# 모델 변경 및 약간의 전처리
- XGBRegressor 변경
- 날짜 분리
- Second: 246 (1280.75201)

In [157]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
# from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [158]:
# !pip3 install xgboost
# !pip3 uninstall lightgbm -y

### data load

In [159]:
df1 = pd.read_csv("../dataset/international_trade.csv")
df2 = pd.read_csv("../dataset/sample_submission.csv")
df3 = pd.read_csv("../dataset/test.csv")
df4 = pd.read_csv("../dataset/train.csv")

## preprocessing

### 수치화

In [160]:
df4 = df4.drop('ID',axis = 1)

In [161]:
def categoryChange(df):
    """ 카테고리화 """
    ordinal_encoder = OrdinalEncoder()
    ordinal = ordinal_encoder.fit_transform(df)
    return ordinal

In [162]:
for name in ['item','corporation','location']:
    df4.loc[:,name] = categoryChange(df4[[name]])
    df4[name] = pd.to_numeric(df4[name])

### 날짜 분리

In [163]:
df4['timestamp'] = pd.to_datetime(df4['timestamp'])
def dateChange(df):
    df.loc[:,'year'] = df['timestamp'].dt.year
    df.loc[:,'month'] = df['timestamp'].dt.month
    df.loc[:,'day'] = df['timestamp'].dt.day
    df.drop('timestamp',axis = 1, inplace = True)
    return df 

In [164]:
df4 = dateChange(df4)

### model training

In [165]:
X = df4.drop(['price(원/kg)','supply(kg)'],axis = 1)
Y = df4['price(원/kg)']

In [166]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42,stratify= df4['item'])

In [167]:
model = RandomForestRegressor()

In [168]:
model = xgb.XGBRegressor()

In [169]:
model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [170]:
pred = model.predict(x_test)

In [171]:
"""
LinearRegressor -> 1904.0070927607574
RandomForestRegressor -> 1580.9442694709762
xgbregressor -> 1580.9064095937372

RandomForestRegressor -> 1192.357782768199 (날짜 전처리)
xgbregressor -> 1148.0481545075852 (날짜 전처리)
"""
forest_mse = mean_squared_error(y_test, pred)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

1148.0481545075852

### Submission

In [172]:
for name in ['item','corporation','location']:
    df3.loc[:,name] = categoryChange(df3[[name]])
    df3[name] = pd.to_numeric(df3[name])

In [173]:
df3['timestamp'] = pd.to_datetime(df3['timestamp'])
df3 = dateChange(df3)

In [174]:
X = df3.drop(['ID'],axis = 1)

In [175]:
real = model.predict(X)

In [176]:
df2

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [177]:
df2.loc[:,'answer'] = real

In [178]:
df2['answer']

0       3183.660889
1       2218.040283
2       1619.587402
3       3205.671143
4       3774.634033
           ...     
1087     499.533447
1088     704.366150
1089      96.116341
1090     978.899475
1091    1142.099731
Name: answer, Length: 1092, dtype: float32

In [179]:
df2.to_csv("answer.csv",index = False)